In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import models
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keract import get_activations
from keras import backend as K

2023-03-28 14:00:50.776841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 14:00:50.977826: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-28 14:00:51.015176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 14:00:51.015193: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
# Read in dataset file and shuffle

ton_iot = pd.read_csv("/home/lolol/Uni/Bachelor/Data/Train_Test_Network.csv")
# ton_iot = ton_iot.sample(frac = 1)
ton_iot.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [3]:
# Get target variable

labels = ton_iot.pop("label")

In [4]:
# Remove unnecessary columns

ton_iot = ton_iot.drop(["ts", "src_ip", "dst_ip", "src_port", "dst_port", "dns_query"], axis=1)
ton_iot.head()

,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,type
0,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,normal
1,udp,-,0.000000,0,0,S0,0,1,63,0,...,0,0,0,-,-,-,-,-,-,normal
2,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,bad_TCP_checksum,-,F,normal
3,tcp,-,0.000000,0,0,OTH,0,0,0,0,...,0,0,0,-,-,-,-,-,-,normal
4,udp,dns,0.000549,0,298,SHR,0,0,0,2,...,0,0,0,-,-,-,bad_UDP_checksum,-,F,normal


In [5]:
# Normalize the numeric values and one hot encoding non numeric fields.

numeric_inputs = []
non_numeric_inputs = []

for name, val in ton_iot.items():
    dtype = val.dtype
    if dtype == object:
        non_numeric_inputs.append(name)
    else:
        numeric_inputs.append(name)
    continue

scaler = MinMaxScaler()

ton_iot[numeric_inputs] = scaler.fit_transform(ton_iot[numeric_inputs])

encoder = LabelEncoder()
for input in non_numeric_inputs:
    ton_iot[input] = encoder.fit_transform(ton_iot[input])

# ton_iot = pd.get_dummies(ton_iot, columns=non_numeric_inputs)

In [6]:
# Splitting the dataset into train and test datasets

x_train, x_test, y_train, y_test = train_test_split(ton_iot, labels, test_size=0.2)
x_train, x_test, y_train, y_test = x_train.to_numpy(), x_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

len(x_train)

368834

In [7]:
x_train[7]

array([1.00000000e+00, 0.00000000e+00, 8.55460083e-11, 0.00000000e+00,
       0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 3.96540580e-06,
       8.44465357e-07, 3.18529160e-06, 4.62986954e-07, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.00000000e+00])

In [8]:
# Build the model

model = keras.Sequential(
    [
        layers.Embedding(output_dim=100, input_dim=100, mask_zero=True),
        layers.Bidirectional(layers.LSTM(50, return_sequences=True)),
        layers.Dropout(0.2),
        layers.Bidirectional(layers.LSTM(50, activation="relu", return_sequences=False)),
        layers.Dropout(0.2),
        layers.Dense(50, activation="linear"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="softmax")
    ]
)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

2023-03-28 14:00:56.878863: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-28 14:00:56.879085: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-28 14:00:56.879122: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lolol-Yoga-7-15ITL5): /proc/driver/nvidia/version does not exist
2023-03-28 14:00:56.879673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Train the model

# model.fit(x=x_train, y=y_train, epochs=1)

In [10]:
# Save the model
# model.save("temp_model")

# Load model from save
model = keras.models.load_model("temp_model")

2023-03-28 14:01:00.042808: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2023-03-28 14:01:00.050594: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2023-03-28 14:01:00.167198: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2023-03-28 14:01:00.325332: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2023-03-28 14:01:00.333414: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_sh

In [11]:
# Evaluate the model

# score = model.evaluate(x_test, y_test, verbose=1)

In [32]:
# Create and test the function for neuron activation coverage
def Neuron_Activation_Coverage(model, input):
    
    if len(input.shape) < 2:
        input = np.expand_dims(input, axis=0)

    activations = get_activations(model, input, auto_compile=True)
    del activations["embedding_input"]
    
    total_nodes = 0
    non_zeros = 0
    for value in activations.values():
        total_nodes += value.size
        non_zeros += np.count_nonzero(value)

    return non_zeros / total_nodes

nac = Neuron_Activation_Coverage(model, x_test)

print("NAC: ", nac)

: 

: 

In [26]:
###### NAC Alternative

# inp = x_test[1]
# inp = np.expand_dims(inp, axis=0)

# from tensorflow.keras import backend as K
# nodes = 0
# non_zero_nodes = 0
# for layerIndex, layer in enumerate(model.layers):
#     func = K.function([model.get_layer(index=0).input], layer.output)
#     layerOutput = func([[inp]])  # input_data is a numpy array
#     nodes += layerOutput.size
#     non_zero_nodes += np.count_nonzero(layerOutput)

# neuronac = non_zero_nodes / nodes

# print(neuronac)

0.4080847790787112


In [27]:
###### NAC Alternative 2

# test_input = x_test[1]
# test_input = np.expand_dims(test_input, axis=0)


# nodes = 0
# non_zero_nodes = 0

# for n in range(0, len(model.layers)):
#     get_nth_layer_output = K.function([model.layers[0].input], [model.layers[n].output])
#     layer_output = get_nth_layer_output([test_input])[0]
#     nodes += layer_output.size

#     non_zero_nodes += np.count_nonzero(layer_output)


# neuron_coverage = non_zero_nodes / nodes

# print("Neuron coverage:", neuron_coverage)

(92209, 38)
(38,)
(1, 38)
Neuron coverage: 0.4080847790787112
